<a href="https://colab.research.google.com/github/JyothyVariyampat/Benchmark-ML-and-DL-Models-for-Small-Molecule-Immunity-Target-Bioactivity-Prediction./blob/main/CNN_AID932.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Deepclassifier-aid1239_revised.ipynb"""

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, cohen_kappa_score, precision_score, recall_score

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

from google.colab import drive


In [ ]:
# Ensure reproducibility
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Load the dataset
#df = pd.read_csv('/content/drive/MyDrive/Datasets/AID_1239/AID1239_extracted_descriptor_data.csv')
#/content/drive/MyDrive/Datasets/AID_1239/concatenated_AID_1239_Active_Inactive.csv
df = pd.read_csv('/content/drive/MyDrive/Datasets/AID_932/concatenated_AID932_4000_Active_Inactive.csv')
df_copy = df.copy()


<ipython-input-42-99a5665a48be>:4: DtypeWarning: Columns (7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,26

In [ ]:
df.shape

(4000, 23530)

In [ ]:
# EDA 1
# Calculate the percentage of NaN values in each column


nan_percentage = df.isna().mean() * 100

# Find columns where more than 50% of the values are NaN
columns_with_nan_above_50 = nan_percentage[nan_percentage > 50].index.tolist()

print(f"Number of columns with more than 50% NaN values: {len(columns_with_nan_above_50)}")
print("Columns:", columns_with_nan_above_50)


Number of columns with more than 50% NaN values: 7594
Columns: ['morgan_counts_207882297', 'morgan_counts_1457862730', 'morgan_counts_3020080207', 'morgan_counts_4021169236', 'morgan_counts_1575699570', 'morgan_counts_1630136439', 'morgan_counts_767018105', 'morgan_counts_2817941632', 'morgan_counts_1858802826', 'morgan_counts_2258556052', 'morgan_counts_311492765', 'morgan_counts_3908726946', 'morgan_counts_1152251050', 'morgan_counts_3182169266', 'morgan_counts_1471496409', 'morgan_counts_728617177', 'morgan_counts_2873572570', 'morgan_counts_302167258', 'morgan_counts_3899674846', 'morgan_counts_705510630', 'morgan_counts_3482235114', 'morgan_counts_2296551640', 'morgan_counts_3766531363', 'morgan_counts_1337673014', 'morgan_counts_671971639', 'morgan_counts_1383003454', 'morgan_counts_3703165273', 'morgan_counts_3507146073', 'morgan_counts_3021899100', 'morgan_counts_2719569261', 'morgan_counts_2106406292', 'morgan_counts_1876824477', 'morgan_counts_2012326308', 'morgan_counts_3856

In [ ]:
# EDA 2
# Calculate the percentage of zero values in each column



zero_percentage = (df == 0).mean() * 100

# Find columns where more than 50% of the values are 0
columns_with_zero_above_40 = zero_percentage[zero_percentage > 40].index.tolist()

print(f"Number of columns with more than 40% zero values: {len(columns_with_zero_above_40)}")
print("Columns:", columns_with_zero_above_40)


Number of columns with more than 40% zero values: 4175
Columns: ['rdf26', 'rdf27', 'rdf28', 'rdf29', 'rdf55', 'rdf56', 'rdf57', 'rdf58', 'rdf59', 'rdf86', 'rdf87', 'rdf88', 'rdf89', 'rdf116', 'rdf117', 'rdf118', 'rdf119', 'rdf146', 'rdf147', 'rdf148', 'rdf149', 'rdf176', 'rdf177', 'rdf178', 'rdf179', 'rdf206', 'rdf207', 'rdf208', 'rdf209', 'maccs0', 'maccs1', 'maccs2', 'maccs3', 'maccs4', 'maccs5', 'maccs6', 'maccs7', 'maccs8', 'maccs9', 'maccs10', 'maccs11', 'maccs12', 'maccs13', 'maccs14', 'maccs15', 'maccs16', 'maccs17', 'maccs18', 'maccs19', 'maccs20', 'maccs21', 'maccs22', 'maccs23', 'maccs24', 'maccs25', 'maccs26', 'maccs27', 'maccs28', 'maccs29', 'maccs30', 'maccs31', 'maccs32', 'maccs33', 'maccs34', 'maccs35', 'maccs36', 'maccs37', 'maccs38', 'maccs39', 'maccs40', 'maccs41', 'maccs42', 'maccs43', 'maccs44', 'maccs45', 'maccs46', 'maccs47', 'maccs48', 'maccs49', 'maccs50', 'maccs51', 'maccs52', 'maccs53', 'maccs54', 'maccs55', 'maccs56', 'maccs57', 'maccs58', 'maccs59', 'maccs60

In [ ]:
# Remove all the columns which falls in this categories:
# Number of columns with more than 50% NaN values:
# Number of columns with more than 50% zero values:

import pandas as pd

# Assuming 'data' is your DataFrame

# 1. Identify columns with more than 50% NaN values
threshold_nan = len(df) * 0.5
columns_with_nan = df.columns[df.isna().sum() > threshold_nan]

# 2. Identify columns with more than 50% zero values
threshold_zero = len(df) * 0.5
columns_with_zero = df.columns[(df == 0).sum() > threshold_zero]

# 3. Combine both sets of columns to drop
columns_to_drop = set(columns_with_nan).union(set(columns_with_zero))

# 4. Drop these columns from the DataFrame
data_cleaned = df.drop(columns=columns_to_drop)

# Check the shape of the new DataFrame to confirm the columns have been removed
print(f"Original shape: {df.shape}")
print(f"New shape after dropping columns: {data_cleaned.shape}")


Original shape: (4000, 23530)
New shape after dropping columns: (4000, 12686)


In [ ]:
df = data_cleaned.copy()

In [ ]:
# Step 1: Fill null values with zero
df.fillna(0, inplace=True)
df = df.round(3)


In [ ]:
# # Save the preprocessed dataset for future reference
# df.to_csv('/content/drive/MyDrive/Datasets/AID_1239/preprocessed_AID1239_12531Columns_Active_Inactive.csv', index=False)
# print(df.head())

In [ ]:
df.head()

,Unnamed: 0,PUBCHEM_CID,PUBCHEM_SID,SMILES,PUBCHEM_ACTIVITY_OUTCOME,SMILES.1,MOLECULEID,autocorr2d0,autocorr2d1,autocorr2d2,...,"atom_pairs_((F,1,0),17,(*,1,0))","bpf_((B,1,0),9,(B,5,0))",morgan_counts_972094048,"atom_pairs_((N,3,0),11,(N,2,1))",morgan_counts_943520092,"atom_pairs_((C,4,0),3,(N,1,2))","atom_pairs_((C,4,0),1,(C,2,2))","atom_pairs_((C,4,0),2,(C,2,2))","atom_pairs_((C,2,2),2,(*,1,0))","atom_pairs_((N,3,0),7,(S,4,0))"
0,0,11840107,22405105,C1CC(OC1)C(=O)NC2=CC3=C(C=C2)OC(=N3)C4=CC=CC=C4,Active,O=C(NC1=CC2=C(OC(=N2)C2=CC=CC=C2)C=C1)C1CCCO1,M3485290,3.424,3.791,3.727,...,0,0,0,0,0,0,0,0,0,0
1,1,2938005,17433378,CC1=CC2=C(C=C1)N=C(S2)C3=CC=C(C=C3)NC(=O)C(C)N...,Active,CC(N1C(=O)C2=CC=CC=C2C1=O)C(=O)NC1=CC=C(C=C1)C...,M3477788,3.787,4.224,4.292,...,0,0,0,0,0,0,0,0,0,0
2,2,2741379,4241533,C1CCN(CC1)C2=NC3=C(S2)C=C(C=C3)Cl,Active,ClC1=CC2=C(C=C1)N=C(S2)N1CCCCC1,M1237690,3.266,3.708,3.645,...,0,0,0,0,0,0,0,0,0,0
3,3,309534,24787233,C1=CC=C(C=C1)OCC(=O)NC2=CC=CC=C2Cl,Active,ClC1=CC=CC=C1NC(=O)COC1=CC=CC=C1,M1191144,3.191,3.508,3.558,...,0,0,0,0,0,0,0,0,0,0
4,4,644550,842291,CCC1=CC2=C(C=C1)OC(=N2)C3=C(C=CC(=C3)N)Cl,Active,CCC1=CC2=C(OC(=N2)C2=C(Cl)C=CC(N)=C2)C=C1,M796259,3.267,3.666,3.716,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Step 1: Replace infinite values with a large finite value
df.replace([np.inf, -np.inf], np.finfo(np.float64).max, inplace=True)

# Step 2: Replace hyphens with NaN
df.replace('-', np.nan, inplace=True)


In [ ]:
# Define target and columns to drop
target_column = 'PUBCHEM_ACTIVITY_OUTCOME'
drop_columns = ['Unnamed: 0','SMILES.1','PUBCHEM_SID', 'PUBCHEM_CID', 'SMILES', 'MOLECULEID']


In [ ]:
# Drop unnecessary columns
df.drop(columns=drop_columns, inplace=True)


In [ ]:
# Encode target column (Active/Inactive)
label_encoder = LabelEncoder()
df[target_column] = label_encoder.fit_transform(df[target_column])

In [ ]:
# Split the features and target
X = df.drop(columns=[target_column]).values
y = df[target_column].values

In [ ]:
# prompt: convert all the X to float


# Convert all columns in X to numeric, coercing errors to NaN
X = pd.DataFrame(X).apply(pd.to_numeric, errors='coerce').values

# Fill NaN values resulting from the conversion (if any)
X = np.nan_to_num(X)

In [ ]:
# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Ensure X_train and X_test are of numeric type before checking for infinity
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

In [ ]:
# Check for NaN and infinity values, replace if needed
X_train[np.isinf(X_train)] = np.nan
X_train = np.nan_to_num(X_train, nan=np.nanmax(X_train))

X_test[np.isinf(X_test)] = np.nan
X_test = np.nan_to_num(X_test, nan=np.nanmax(X_test))

In [ ]:
# Step 3: Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Define the CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1), kernel_regularizer=l2(0.001)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# Compile the model
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Early stopping
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, callbacks=[early_stopping])


Epoch 1/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 52ms/step - accuracy: 0.6519 - loss: 0.7724 - val_accuracy: 0.7425 - val_loss: 0.6847
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7997 - loss: 0.6037 - val_accuracy: 0.7700 - val_loss: 0.6365
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8333 - loss: 0.5265 - val_accuracy: 0.7738 - val_loss: 0.6106
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8616 - loss: 0.4716 - val_accuracy: 0.7825 - val_loss: 0.5918
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8846 - loss: 0.4224 - val_accuracy: 0.7825 - val_loss: 0.6013
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9035 - loss: 0.3809 - val_accuracy: 0.7575 - val_loss: 0.6525
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9162 - loss: 0.3451 - val_accuracy: 0.7900 - val_loss: 0.6160


In [ ]:
# Evaluate the model
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)



In [ ]:
# Make predictions
y_train_pred = model.predict(X_train).flatten()
y_train_pred_classes = (y_train_pred > 0.5).astype(int)
y_pred = model.predict(X_test).flatten()
y_pred_classes = (y_pred > 0.5).astype(int)


100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [ ]:
# Calculate metrics for training data
train_roc_auc = roc_auc_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred_classes)
train_kappa = cohen_kappa_score(y_train, y_train_pred_classes)
train_precision = precision_score(y_train, y_train_pred_classes)
train_recall = recall_score(y_train, y_train_pred_classes)


In [ ]:
# Calculate metrics for testing data
test_roc_auc = roc_auc_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred_classes)
test_kappa = cohen_kappa_score(y_test, y_pred_classes)
test_precision = precision_score(y_test, y_pred_classes)
test_recall = recall_score(y_test, y_pred_classes)

In [ ]:
# Output the results
results = {
    "train_accuracy": train_acc,
    "train_roc_auc": train_roc_auc,
    "train_f1": train_f1,
    "train_kappa": train_kappa,
    "train_precision": train_precision,
    "train_recall": train_recall,
    "test_accuracy": test_acc,
    "test_roc_auc": test_roc_auc,
    "test_f1": test_f1,
    "test_kappa": test_kappa,
    "test_precision": test_precision,
    "test_recall": test_recall
}

print("\nModel Evaluation Metrics:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")


Model Evaluation Metrics:
train_accuracy: 0.9062
train_roc_auc: 0.9670
train_f1: 0.9108
train_kappa: 0.8123
train_precision: 0.8799
train_recall: 0.9439
test_accuracy: 0.7825
test_roc_auc: 0.8674
test_f1: 0.7868
test_kappa: 0.5673
test_precision: 0.7329
test_recall: 0.8492


In [ ]:
# Confusion matrix and classification report
cm = confusion_matrix(y_test, y_pred_classes)
print("\nConfusion Matrix:")
print(cm)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))


Confusion Matrix:
[[305 117]
 [ 57 321]]

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.72      0.78       422
           1       0.73      0.85      0.79       378

    accuracy                           0.78       800
   macro avg       0.79      0.79      0.78       800
weighted avg       0.79      0.78      0.78       800



1. Training Accuracy: 0.9062 (90.62%)
Interpretation: This indicates that your model is able to correctly classify 90.62% of the samples in the training set. A high training accuracy suggests that the model has learned well on the training data. However, if the training accuracy is significantly higher than the test accuracy, it might indicate overfitting.
2. Test Accuracy: 0.7825 (78.25%)
Interpretation: This is the overall accuracy on the test dataset. It means that 78.25% of the test samples were classified correctly. While this is lower than the training accuracy, it’s still reasonably high. The difference between training and test accuracy indicates some level of overfitting, but not extreme.
3. ROC-AUC Score: 0.8674
Interpretation: The ROC-AUC score measures the ability of the model to distinguish between the positive and negative classes. A score of 0.8674 is good (close to 1), indicating that the model has a strong capability of differentiating between active and inactive classes in the test data. This is a positive sign of the model's performance.
4. F1 Score: 0.7868
Interpretation: The F1 score is the harmonic mean of precision and recall. It balances the trade-off between these two metrics:
A score of 0.7868 suggests that the model's precision and recall are balanced well.
It indicates a strong overall performance, especially in scenarios where the dataset might be imbalanced.
5. Cohen’s Kappa: 0.5673
Interpretation: Cohen's Kappa measures the agreement between predicted and actual classifications while accounting for the possibility of agreement by chance.
A Kappa score of 0.5673 indicates moderate agreement between the predictions and the true labels.
It suggests that while the model performs well, there is still some room for improvement in the consistency of predictions.
6. Precision: 0.7329
Interpretation: Precision indicates the proportion of true positive predictions among all positive predictions made by the model.
A precision of 0.7329 means that when the model predicts a sample as "active," it is correct about 73.29% of the time.
This shows that the model has a relatively low rate of false positives.
7. Recall: 0.8492
Interpretation: Recall (or Sensitivity) measures the proportion of actual positive cases that were correctly identified by the model.
A recall score of 0.8492 means that the model correctly identifies 84.92% of the actual active samples.
This high recall suggests that the model is good at capturing the majority of positive (active) cases, minimizing false negatives.
Summary and Recommendations:
The model shows strong performance, with good training and test accuracy, high ROC-AUC, and a balanced F1 score.
The difference between training (90.62%) and test accuracy (78.25%) suggests a bit of overfitting. You might consider techniques such as:
Adding regularization (e.g., L2 regularization or dropout).
Collecting more data if possible or applying data augmentation techniques.
Using early stopping to prevent overfitting during training.
The relatively high ROC-AUC and recall scores indicate that the model is effective at distinguishing between the two classes and minimizing false negatives, which is often important in classification tasks related to drug discovery (e.g., detecting active compounds).
The moderate Cohen’s Kappa score (0.5673) suggests room for improvement in the model’s agreement with the ground truth, potentially by further optimizing hyperparameters.

no optuna ws run for aid 932


**Below is Code with Optuna**

In [ ]:
!pip install --quiet optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.4 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, cohen_kappa_score, precision_score, recall_score
from tensorflow.keras.callbacks import EarlyStopping

# Define the Optuna objective function
def objective(trial):
    # Hyperparameter suggestions
    n_layers = trial.suggest_int("n_layers", 2, 4)
    units = trial.suggest_int("units", 32, 256, step=32)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)
    l2_reg = trial.suggest_float("l2_reg", 1e-5, 1e-2, log=True)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    # Build the model
    model = Sequential()
    model.add(Dense(units, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(l2_reg)))
    model.add(Dropout(dropout_rate))
    for _ in range(n_layers - 1):
        model.add(Dense(units // 2, activation='relu', kernel_regularizer=l2(l2_reg)))
        model.add(Dropout(dropout_rate))
        units //= 2
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
                        batch_size=batch_size, callbacks=[early_stopping], verbose=0)

    # Evaluate the model on test data
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

    # Calculate additional metrics on test data
    y_pred = model.predict(X_test).flatten()
    y_pred_classes = (y_pred > 0.5).astype(int)

    if len(np.unique(y_test)) < 2:
        test_roc_auc = None
    else:
        test_roc_auc = roc_auc_score(y_test, y_pred)

    test_f1 = f1_score(y_test, y_pred_classes)
    test_kappa = cohen_kappa_score(y_test, y_pred_classes)
    test_precision = precision_score(y_test, y_pred_classes)
    test_recall = recall_score(y_test, y_pred_classes)

    # Return the test accuracy as the objective to maximize
    return test_acc

# Run Optuna for hyperparameter optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

# Best hyperparameters from Optuna
best_params = study.best_params
print("Best hyperparameters:", best_params)

# Build the final model with the best hyperparameters
n_layers = best_params["n_layers"]
units = best_params["units"]
dropout_rate = best_params["dropout_rate"]
l2_reg = best_params["l2_reg"]
learning_rate = best_params["learning_rate"]
batch_size = best_params["batch_size"]

model = Sequential()
model.add(Dense(units, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(l2_reg)))
model.add(Dropout(dropout_rate))
for _ in range(n_layers - 1):
    model.add(Dense(units // 2, activation='relu', kernel_regularizer=l2(l2_reg)))
    model.add(Dropout(dropout_rate))
    units //= 2
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the final model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
                    batch_size=batch_size, callbacks=[early_stopping], verbose=1)

# Evaluate the final model
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

# Predictions
y_train_pred = model.predict(X_train).flatten()
y_train_pred_classes = (y_train_pred > 0.5).astype(int)
y_pred = model.predict(X_test).flatten()
y_pred_classes = (y_pred > 0.5).astype(int)

# Train metrics
train_roc_auc = roc_auc_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred_classes)
train_kappa = cohen_kappa_score(y_train, y_train_pred_classes)
train_precision = precision_score(y_train, y_train_pred_classes)
train_recall = recall_score(y_train, y_train_pred_classes)

# Test metrics
test_roc_auc = roc_auc_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred_classes)
test_kappa = cohen_kappa_score(y_test, y_pred_classes)
test_precision = precision_score(y_test, y_pred_classes)
test_recall = recall_score(y_test, y_pred_classes)

# Output the results
results = {
    "train_accuracy": train_acc,
    "train_roc_auc": train_roc_auc,
    "train_f1": train_f1,
    "train_kappa": train_kappa,
    "train_precision": train_precision,
    "train_recall": train_recall,
    "test_accuracy": test_acc,
    "test_roc_auc": test_roc_auc,
    "test_f1": test_f1,
    "test_kappa": test_kappa,
    "test_precision": test_precision,
    "test_recall": test_recall
}

print("\nModel Evaluation Metrics:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")


[I 2024-11-17 15:25:28,131] A new study created in memory with name: no-name-b1fce05e-2300-46a2-9a9c-e5935f2c0eb0
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-11-17 15:25:51,997] Trial 0 finished with value: 0.5274999737739563 and parameters: {'n_layers': 4, 'units': 160, 'dropout_rate': 0.5, 'l2_reg': 0.001849274503033483, 'learning_rate': 2.652215497829439e-05, 'batch_size': 64}. Best is trial 0 with value: 0.5274999737739563.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


[I 2024-11-17 15:26:09,212] Trial 1 finished with value: 0.4724999964237213 and parameters: {'n_layers': 4, 'units': 64, 'dropout_rate': 0.4, 'l2_reg': 0.0004973452654829904, 'learning_rate': 0.0002902165965035215, 'batch_size': 128}. Best is trial 0 with value: 0.5274999737739563.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step   


[I 2024-11-17 15:32:34,713] Trial 29 finished with value: 0.4724999964237213 and parameters: {'n_layers': 4, 'units': 224, 'dropout_rate': 0.4, 'l2_reg': 0.00053242208819626, 'learning_rate': 0.00026926646935074565, 'batch_size': 128}. Best is trial 0 with value: 0.5274999737739563.


Best hyperparameters: {'n_layers': 4, 'units': 160, 'dropout_rate': 0.5, 'l2_reg': 0.001849274503033483, 'learning_rate': 2.652215497829439e-05, 'batch_size': 64}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - accuracy: 0.5061 - loss: 1.7378 - val_accuracy: 0.4725 - val_loss: 1.5900
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4788 - loss: 1.7222 - val_accuracy: 0.5275 - val_loss: 1.5749
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4968 - loss: 1.6604 - val_accuracy: 0.4725 - val_loss: 1.5679
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5101 - loss: 1.6952 - val_accuracy: 0.4725 - val_loss: 1.5629
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5168 - loss: 1.5651 - val_accuracy: 0.4725 - val_loss: 1.5581
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5020 - loss: 1.8170 - val_accuracy: 0.4725 - val_loss: 1.5541
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5003 - loss: 1.6166 - val_accuracy: 0.4725 - val_loss: 1.5502
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5177 - loss: 1.8011 - val_accuracy: 0.4725 - val_loss

below code with old ones

In [ ]:
# Step 4: Build the model
model = Sequential()


In [ ]:
# Input layer with L2 regularization
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))  # Dropout for regularization


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Hidden layers with L2 regularization
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))

model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))

In [ ]:
# Output layer (binary classification)
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# Compile the model with Adam optimizer and binary cross-entropy loss
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Add callbacks for early stopping and model checkpoint
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
# Early stopping to prevent overfitting, model checkpoint to save the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Jyothi Research Paper/best_model_fulldescriptors_aid1239.keras', save_best_only=True)


In [ ]:
# Step 5: Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping, model_checkpoint])

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 76ms/step - accuracy: 0.6455 - loss: 4.4548 - val_accuracy: 0.7788 - val_loss: 3.5156
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7659 - loss: 3.3761 - val_accuracy: 0.7987 - val_loss: 2.7593
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8290 - loss: 2.5753 - val_accuracy: 0.7950 - val_loss: 2.1734
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8600 - loss: 1.9603 - val_accuracy: 0.8050 - val_loss: 1.8138
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8813 - loss: 1.6092 - val_accuracy: 0.8050 - val_loss: 1.6028
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8898 - loss: 1.4058 - val_accuracy: 0.8200 - val_loss: 1.4967
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9102 - loss: 1.2482 - val_accuracy: 0.8125 - val_loss: 1.4136
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9196 - loss: 1.1415 - val_accuracy: 0.7850 - val_loss

In [ ]:
# Step 6: Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8195 - loss: 0.9831 
Test Accuracy: 81.12%


In [ ]:
# Step 7: Make predictions
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)  # Use 0.5 as threshold for binary classification


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


In [ ]:
# Assuming you have 'df_copy' (original dataframe) and 'y_pred_classes' available

# Predict on the entire dataset to get predictions for all rows
# (Assuming 'X' contains features for all rows in 'df_copy')
y_pred_all = model.predict(X)
y_pred_classes_all = (y_pred_all > 0.5).astype(int)

# Create a new DataFrame with the specified columns
new_df = pd.DataFrame()

# Assuming 'df_copy' contains 'PUBCHEM_CID' and 'PUBCHEM_ACTIVITY_OUTCOME'
new_df['PUBCHEM_CID'] = df_copy['PUBCHEM_CID']  # Use values from the original df
new_df['PUBCHEM_ACTIVITY_OUTCOME'] = df_copy['PUBCHEM_ACTIVITY_OUTCOME'] # Use values from original df

# Add the predicted classes for all rows and map to "Active"/"Inactive"
new_df['y_pred_classes'] = y_pred_classes_all.flatten()
new_df['y_pred_classes'] = new_df['y_pred_classes'].map({1: 'Active', 0: 'Inactive'}) # Map 1 to 'Active' and 0 to 'Inactive'

# Display or save the new DataFrame as needed
print(new_df.head())
#new_df.to_csv('predicted_outcomes.csv', index=False)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
   PUBCHEM_CID PUBCHEM_ACTIVITY_OUTCOME y_pred_classes
0      1449342                   Active       Inactive
1      3242114                   Active       Inactive
2      9594900                   Active       Inactive
3       823601                   Active       Inactive
4      3239831                   Active       Inactive


In [ ]:
# Step 8: Evaluate the results
compound_ids = df_copy['PUBCHEM_CID'].values
compound_ids_train, compound_ids_test = train_test_split(compound_ids, test_size=0.2, random_state=42)


In [ ]:
results = [
   f"Compound {compound_id}: This is active" if pred == 1 else f"Compound {compound_id}: This is inactive"
   for compound_id, pred in zip(compound_ids_test, y_pred_classes.flatten())
]

results

['Compound 2986465: This is inactive',
 'Compound 1319487: This is active',
 'Compound 2999802: This is inactive',
 'Compound 16190573: This is active',
 'Compound 666181: This is active',
 'Compound 135472859: This is inactive',
 'Compound 711992: This is inactive',
 'Compound 6898097: This is inactive',
 'Compound 2986519: This is inactive',
 'Compound 135490211: This is inactive',
 'Compound 1867778: This is active',
 'Compound 5762397: This is active',
 'Compound 1481967: This is inactive',
 'Compound 2831188: This is active',
 'Compound 2515723: This is active',
 'Compound 4120298: This is active',
 'Compound 1481828: This is inactive',
 'Compound 684082: This is inactive',
 'Compound 5730347: This is inactive',
 'Compound 3237354: This is inactive',
 'Compound 752122: This is inactive',
 'Compound 5721129: This is inactive',
 'Compound 5724809: This is active',
 'Compound 755827: This is inactive',
 'Compound 4248181: This is active',
 'Compound 704968: This is inactive',
 'Compo

In [ ]:
# Confusion matrix and classification report
cm = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[332  90]
 [ 61 317]]


In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))



Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.79      0.81       422
           1       0.78      0.84      0.81       378

    accuracy                           0.81       800
   macro avg       0.81      0.81      0.81       800
weighted avg       0.81      0.81      0.81       800



In [ ]:
# Optional: Save the model for future use
model.save('/content/drive/MyDrive/Jyothi Research Paper/final_deep_classifier_model.keras')


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, cohen_kappa_score, precision_score, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import random

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Load your dataset
df = pd.read_csv('/content/drive/MyDrive/Datasets/AID_1239/concatenated_AID_1239_Active_Inactive.csv')

# Drop unnecessary columns
columns_to_drop = ['Unnamed: 0', 'PUBCHEM_CID', 'PUBCHEM_SID', 'SMILES', 'SMILES.1', 'MOLECULEID']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Encode target column
df['PUBCHEM_ACTIVITY_OUTCOME'] = df['PUBCHEM_ACTIVITY_OUTCOME'].apply(lambda x: 1.0 if x == 'Active' else 0.0)

# Split features and target
X = df.drop(columns=['PUBCHEM_ACTIVITY_OUTCOME']).values
y = df['PUBCHEM_ACTIVITY_OUTCOME'].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64, callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

# Make predictions
y_pred = model.predict(X_test).flatten()
y_pred_classes = (y_pred > 0.5).astype(int)

# Calculate metrics
test_roc_auc = roc_auc_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred_classes)
test_kappa = cohen_kappa_score(y_test, y_pred_classes)
test_precision = precision_score(y_test, y_pred_classes)
test_recall = recall_score(y_test, y_pred_classes)

# Output the results
results = {
    "accuracy": test_acc,
    "roc_auc": test_roc_auc,
    "f1": test_f1,
    "kappa": test_kappa,
    "precision": test_precision,
    "recall": test_recall
}

print("Model Evaluation Metrics:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")


ValueError: Input X contains infinity or a value too large for dtype('float64').